In [1]:
#!pip install google.colab
!pip install rouge_score

ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.15.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0


In [4]:
import requests        # for making http requests to binance
import json            # for parsing what binance sends back to us
import pandas as pd    # for storing and manipulating the data we get back
import numpy as np     # numerical python, i usually need this somewhere 
import matplotlib.pyplot as plt # for charts and such
import json
import nltk
import csv
from sklearn.utils import shuffle
import pandas as pd
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from datetime import timedelta, datetime
from dateutil import parser
from nltk.stem import WordNetLemmatizer
import numpy.random as nr
import tensorflow.keras as keras
#import keras.utils.np_utils as ku
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer, tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

from keras.optimizers import rmsprop, Adam
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
import glob
import gensim
import pandas as pd
import os
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Concatenate,Dropout, GRU, LSTM, Input, Activation, Conv1D, GlobalMaxPooling1D, Input,RepeatVector, Flatten,TimeDistributed,Embedding
from google.colab import drive
drive.mount('colabDrive')
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('wordnet')
path = 'colabDrive/My Drive/Colab Notebooks/LTP/data/'
import pandas as pd
import os
from string import punctuation
from nltk.stem.porter import PorterStemmer

#vocab_size = 20000
oov_tok = '<OOV>'
max_length = 100
def pre_process(data):
  from string import punctuation
  from nltk.stem.porter import PorterStemmer
  processed_data = []
  ps = PorterStemmer()
  word_lemm = WordNetLemmatizer()
  print(len(data))
  for i in range(len(data)):
    doc2Txt = str(data[i])

    txt =  ''.join(c for c in doc2Txt if not c.isdigit())
    txt = ''.join(c for c in txt if c not in punctuation).lower()
    # Get the word stems
    #txt =  ''.join( ps.stem(word) for word in txt)
    txt =  ' '.join([word_lemm.lemmatize(word) for word in txt.split()])
    txt = ' '.join([word for word in txt.split() if word not in (stopwords.words('english'))])
    processed_data.append(txt)
    if i % 2000 == 0:
      print('i = :' + str(i))
  return processed_data

def get_tokenize_sequence(data):
  
  print(vocab_size)
  tokenizer = Tokenizer(oov_token=oov_tok)
  tokenizer.fit_on_texts(data)
  word_index = tokenizer.word_index
  
  return tokenizer
def loadProcessedSequence():
    x_train = np.load('x_train_padded.npy')
    y_train = np.load('y_train_padded.npy')
    return x_train, y_train

def get_tokenizer(data,oov_token ):
  
  tokenizer = Tokenizer(oov_token=oov_token)
  tokenizer.fit_on_texts(data)
  word_index = tokenizer.word_index
  return tokenizer

def addEosTokens(data):
  for i in range(len(data)):
    data[i] = "<start> " + data[i] + " <eos>"
  return data
def loadFiles(index):
    #indexes from 1 to 7 
    count = 0 
    articles_ = np.array([])
    for i in range(1,index+1):
      articles_ = np.append(articles_,(np.load(path+'articles_lem_%d.npy'%i)))
    
    titles_ = np.load(path+'titles_lem_100k.npy')[:len(articles_)]

    
    return articles_,titles_
def getVocabSize(data):
  vocab = set()
  for i in range(len(data)):
    words = data[i].split()
    for word in words:
      vocab.add(word)
  return len(vocab)

def loadTokenizer(filename):    
    with open(path+filename) as f:
        data = json.load(f)
        tokenizer_x = tf.keras.preprocessing.text.tokenizer_from_json(data)
    
   
    return tokenizer_x

def loadWord2vec(data, tok, vocab_size, latent_dim):
    embedding_matrix = np.zeros(
    (vocab_size, 100),
    dtype='float32')
    word2vec = gensim.models.Word2Vec(data, size=latent_dim, min_count=5, iter=10)
    for word, i in tok.word_index.items():
        if word in word2vec.wv.vocab:
            if i < len(embedding_matrix):
              embedding_matrix[i] = word2vec.wv.word_vec(word)
    return word2vec, embedding_matrix
def build_data():
    # read data from the csv file (from the location it is stored)
    Data = pd.read_csv('colabDrive/My Drive/Colab Notebooks/LTP/dataset/wikihowAll.csv')
    Data = Data.astype(str)
    rows, columns = Data.shape
    summaries = []
    articles = []
    titles = []
    # create a file to record the file names. This can be later used to divide the dataset in train/dev/test sets
    #title_file = open('titles.txt', 'wb')
    
    
    for row in range(rows):

        summary = Data.loc[row,'headline']
                          # headline is the column representing the summary sentences
        article = Data.loc[row,'text']           # text is the column representing the article
        title = Data.loc[row,'title']
        #  a threshold is used to remove short articles with long summaries as well as articles with no summary
        if len(summary) < (0.75*len(article)) and len(articles) < 1100:
            # remove extra commas in abstracts
            summary = summary.replace(".,",".")
            summary = summary.encode('utf-8')
            title = title.replace(".,",".")
            title = title.encode('utf-8')
            # remove extra commas in articles
            article = re.sub(r'[.]+[\n]+[,]',".\n", article)
            article = article.encode('utf-8')
            summaries.append(summary.decode())
            articles.append(article.decode())
            titles.append(title.decode())
    return articles, titles
def plot_accuracy(history):
    train_acc = history['acc']
    test_acc = history['val_acc']
    x = list(range(1, len(test_acc) + 1))
    plt.plot(x, test_acc, color = 'red', label = 'test accuracy')
    plt.plot(x, train_acc, label = 'training accuracy')  
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs. Epoch')  
    plt.legend(loc='lower right')
    plt.show()

def plot_loss(history):
    from pylab import rcParams
    rcParams['figure.figsize'] = 5, 5

    train_acc = history['loss']
    test_acc = history['val_loss']
    x = list(range(1, len(test_acc) + 1))
    plt.plot(x, test_acc, color = 'red', label = 'test loss')
    plt.plot(x, train_acc, label = 'training loss')  
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Word2Vec Loss vs. Epoch')  
    plt.legend(loc='lower right')
    plt.show()

print('loaded')

Drive already mounted at colabDrive; to attempt to forcibly remount, call drive.mount("colabDrive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
loaded


In [ ]:
#json.dump(history_dict, open(path+'epoch-hist%d.json'%epoch_index, 'w'))
#model.save_weights(path+'model_weights%d.h5'%epoch_index)


Loading Saved train/test data and related files 

In [ ]:
# encoder inference
latent_dim=100
# encoder inference
def GetEncoderDecoderInferenceModels(model):
    latent_dim=100
    encoder_inputs = model.input[0]  #loading encoder_inputs
    encoder_outputs, state_h, state_c = model.layers[6].output #loading encoder_outputs
    #print(encoder_outputs.shape)
    encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])
    # decoder inference
    # Below tensors will hold the states of the previous time step
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_hidden_state_input = Input(shape=(max_encoder_seq_length,latent_dim))
    # Get the embeddings of the decoder sequence
    decoder_inputs = model.layers[3].output
    #print(decoder_inputs.shape)
    dec_emb_layer = model.layers[5]
    dec_emb2= dec_emb_layer(decoder_inputs)
    # To predict the next word in the sequence, set the initial states to the states from the previous time step
    decoder_lstm = model.layers[7]
    decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])
    #attention inference
    attn_layer = model.layers[8]
    attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
    concate = model.layers[9]
    decoder_inf_concat = concate([decoder_outputs2, attn_out_inf])
    # A dense softmax layer to generate prob dist. over the target vocabulary
    decoder_dense = model.layers[10]
    decoder_outputs2 = decoder_dense(decoder_inf_concat)
    # Final decoder model
    decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])
    return encoder_model,decoder_model

def decode_sequence(input_seq, encoderModel, decoderModel):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoderModel.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Chose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = tok_y.word_index['start']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoderModel.predict([target_seq] + [e_out, e_h, e_c])
        # Sample a token
        #print(output_tokens)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = ''
        if sampled_token_index != 0:
          sampled_token = reverse_word_index_y[sampled_token_index]

        if(sampled_token!='eos'):
            decoded_sentence += ' '+sampled_token

            # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eos' or len(decoded_sentence.split()) >= (max_decoder_seq_length-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence



Loading saved training data and test data 
Preparing input for the model 

In [ ]:
x_data = np.load(path+'x_data.npy')
y_data = np.load(path+'y_data.npy')
idx = np.random.RandomState(seed=42).permutation(x_data.shape[0])
test_size = 0.2 * len(x_data)
test_size = int(test_size)
train_idx = idx[test_size:]
test_idx = idx[:test_size]
x_train = x_data[train_idx]
y_train = y_data[train_idx]
x_test = x_data[test_idx]
y_test = y_data[test_idx]
embedding_x = np.load(path+'embedding_x.npy')
embedding_y = np.load(path+'embedding_y.npy')
tok_x = loadTokenizer('tokenizer_x.json')
tok_y = loadTokenizer('tokenizer_y.json')
reverse_word_index_x = dict([(value, key) for (key, value) in tok_x.word_index.items()])
reverse_word_index_y = dict([(value, key) for (key, value) in tok_y.word_index.items()])




In [ ]:
tok_y = get_tokenizer(titles,oov_token=oov_tok )
y_seq = tok_y.texts_to_sequences(titles)
y_data = pad_sequences(y_seq, maxlen=10)
np.save('colabDrive/My Drive/Colab Notebooks/LTP/data/y_data.npy',y_data)
import io
tokenizer_json = tok_y.to_json()
with io.open('colabDrive/My Drive/Colab Notebooks/LTP/data/tokenizer_y.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))
w2v, embedding_y = loadWord2vec(titles, tok_y, len(tok_y.word_index)+1, 100)
np.save('colabDrive/My Drive/Colab Notebooks/LTP/data/embedding_y.npy',embedding_y)

In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state """

            assert_msg = "States must be a list. However states {} is of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch_size*en_seq_len, latent_dim
            reshaped_enc_outputs = K.reshape(encoder_out_seq, (-1, en_hidden))
            # <= batch_size*en_seq_len, latent_dim
            W_a_dot_s = K.reshape(K.dot(reshaped_enc_outputs, self.W_a), (-1, en_seq_len, en_hidden))
            if verbose:
                print('wa.s>',W_a_dot_s.shape)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>',U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            reshaped_Ws_plus_Uh = K.tanh(K.reshape(W_a_dot_s + U_a_dot_h, (-1, en_hidden)))
            if verbose:
                print('Ws+Uh>', reshaped_Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.reshape(K.dot(reshaped_Ws_plus_Uh, self.V_a), (-1, en_seq_len))
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        def create_inital_state(inputs, hidden_size):
            # We are not using initial states, but need to pass something to K.rnn funciton
            fake_state = K.zeros_like(inputs)  # <= (batch_size, enc_seq_len, latent_dim
            fake_state = K.sum(fake_state, axis=[1, 2])  # <= (batch_size)
            fake_state = K.expand_dims(fake_state)  # <= (batch_size, 1)
            fake_state = K.tile(fake_state, [1, hidden_size])  # <= (batch_size, latent_dim
            return fake_state

        fake_state_c = create_inital_state(encoder_out_seq, encoder_out_seq.shape[-1])
        fake_state_e = create_inital_state(encoder_out_seq, encoder_out_seq.shape[1])  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [ ]:
latent_dim = 100
max_encoder_seq_length = 100
max_decoder_seq_length = 10
num_decoder_tokens = len(tok_x.word_index)+1
num_encoder_tokens = len(tok_y.word_index)+1
from sklearn.utils import shuffle
epochs = 100
import random
x_vocab_size = len(tok_x.word_index)+1
y_vocab_size = len(tok_y.word_index)+1

encoder_input_data = Input(shape=(None, max_encoder_seq_length, None))

decoder_input_data = Input(shape=(None, max_decoder_seq_length, num_decoder_tokens))
#weights=[embedding_x] weights=[embedding_y]
# Encoder 
encoder_inputs = Input(shape=(max_encoder_seq_length,)) 
enc_emb = Embedding(x_vocab_size , latent_dim,trainable=True)(encoder_inputs) 
#kernel_regularizer='l2'
#LSTM 1 
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb) 

#LSTM 2 
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,kernel_regularizer='l2') 
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1) 
#dropoutLayer = Dropout(0.2)
#drop_output = dropoutLayer(encoder_output2)
#LSTM 3 
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True) 
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2) 
# Set up the decoder. 
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(y_vocab_size , latent_dim,trainable=True) 
dec_emb = dec_emb_layer(decoder_inputs) 

#LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c]) 

#Attention Layer
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs]) 

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#temp = Dense(units=100,activation='relu')
##temp_outputs = temp(decoder_concat_input)

#Dense layer
decoder_dense = TimeDistributed(Dense(y_vocab_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs) 
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['acc'])

model.summary()
# Train 


sampleSize = 10000
from rouge_score import rouge_scorer

epoch_index = 0
epochs = 0
#model.load_weights(path+'model_weights120.h5')
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=False)

from tensorflow.keras.callbacks import EarlyStopping

while True:
    model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.001), loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

    history=model.fit([x_train,y_train[:,:-1]], y_train.reshape(y_train.shape[0],y_train.shape[1], 1)[:,1:],epochs = epochs, batch_size=1024,
                 validation_data=([x_test,y_test[:,:-1]], y_test.reshape(y_test.shape[0],y_test.shape[1], 1)[:,1:]))

    epoch_index = epoch_index + epochs
    history_dict = history.history

    percision = []
    recall = []
    
    encoder, decoder = GetEncoderDecoderInferenceModels(model)
    #index = random.randint(1,1000)
    
    ## print score for 100 test articles 
    for i in range (100):
        predicted = decode_sequence(x_test[i].reshape(1,max_encoder_seq_length), encoder, decoder)
        gold = y_test[i]
        gold_output = []
        scores = scorer.score(str(gold),
                             str(predicted))
        for j in range(len(gold)):
          if gold[j] != 0:
            gold_output.append(reverse_word_index_y[gold[j]])
        print('************************************************************************************')
        print("gold: " + str(gold_output))
        print("predicted: " + predicted)
        print('************************************************************************************')

        percision.append(scores['rouge1'][0])
        recall.append(scores['rouge1'][0])
        index = index + 10
    print(percision)

In [ ]:
model.load_weights(path+'baseline-model_weights80.h5')
percision = []
recall = []
percisonL = []
recallL = []
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.001), loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

encoder, decoder = GetEncoderDecoderInferenceModels(model)

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=False)
from_epoch = 0

for i in range (from_epoch,len(x_test)):
        index = random.randint(0,len(x_test)-1)
        predicted = decode_sequence(x_test[i].reshape(1,max_encoder_seq_length), encoder, decoder)
        gold = y_test[i]
        gold_output = []
        gold_str = ''                     
        for j in range(len(gold)):
          if gold[j] != 0:
                gold_str = gold_str +  reverse_word_index_y[gold[j]] + " "

       # print('************************************************************************************')
       # print("gold: " + str(gold_output))
       # print("predicted: " + predicted)
       # print('************************************************************************************')
        scores = scorer.score(gold_str,
                          str(predicted))
        percision.append(scores['rouge1'][0])
        recall.append(scores['rouge1'][1])
        percisonL.append(scores['rougeL'][0])
        recallL.append(scores['rougeL'][1])
        if i % 1000 == 0 or i == len(x_test) - 1 :
          file_name = path+"baseline-score_%d_%d.txt"%(from_epoch,i)
          avgPercision = sum(percision)/len(percision)
          avgRecall = sum(recall)/len(recall)
          avgPercisionL = sum(percisonL)/len(percisonL)
          avgRecallL = sum(recallL)/len(recallL)
          x = (avgPercision,avgRecall,avgPercisionL,avgRecallL,i)
          print(x)
          np.savetxt(file_name,x)
          #x = np.loadtxt(file_name)

        score = "percision: %f, recall: %f,percisionL: %f, recallL: %f , at index %d "%(avgPercision,avgRecall,avgPercisionL,avgRecallL,i)
          
#print(sum(percision)/len(percision),  sum(recall)/len(recall), sum(percisonL)/len(percisonL),sum(recallL)/len(recallL))


(0.125, 0.1, 0.125, 0.1, 0)
(0.33303521874950454, 0.31790356108537904, 0.330551591265877, 0.31575174464784833, 1000)
(0.3303386005409994, 0.3163278389665199, 0.3277753980152781, 0.314089474022008, 2000)
(0.331241305808117, 0.31630145651305475, 0.3280823058980341, 0.3135756455399931, 3000)
(0.3311507440600163, 0.31617869342188004, 0.3276788937686209, 0.31318410476745656, 4000)
(0.3308527183452192, 0.31596827099225977, 0.32745220797110347, 0.31302933496994023, 5000)
(0.3298592430362126, 0.31539205346486493, 0.3265046434059922, 0.3124886352514979, 6000)
(0.3311827592965994, 0.3165299051279533, 0.3278649226156717, 0.3136553724648182, 7000)
(0.33102471588598265, 0.31667464727081496, 0.32777874266438134, 0.3138684504263723, 8000)
(0.33067390459902357, 0.3160376440456371, 0.3274131557933353, 0.31322014405004517, 9000)
(0.33021826605218235, 0.31568393665683503, 0.3270335845203355, 0.3129351242364893, 10000)
(0.330132269885474, 0.3159046737585025, 0.32687185200138547, 0.3130907881521428, 11000)